In [1]:
import calendar
import matplotlib.pyplot as plt
from datetime import datetime
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn.preprocessing import LabelEncoder
import locale
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import association_rules,apriori
from scipy import sparse

In [22]:
user = pd.read_csv('cellphones ratings.csv')

user.head(3)

,user_id,cellphone_id,rating
0,0,30,1
1,0,5,3
2,0,10,9


In [23]:
product = pd.read_csv('cellphones data.csv')

product.head(3)

,cellphone_id,brand,model,operating system,internal memory,RAM,performance,main camera,selfie camera,battery size,screen size,weight,price,release date
0,0,Apple,iPhone SE (2022),iOS,128,4,7.23,12,7,2018,4.7,144,429,18/03/2022
1,1,Apple,iPhone 13 Mini,iOS,128,4,7.72,12,12,2438,5.4,141,699,24/09/2021
2,2,Apple,iPhone 13,iOS,128,4,7.75,12,12,3240,6.1,174,699,24/09/2021


In [8]:
df = pd.merge(user, product, on='cellphone_id')

df

,user_id,cellphone_id,rating,brand,model,operating system,internal memory,RAM,performance,main camera,selfie camera,battery size,screen size,weight,price,release date
0,0,30,1,Motorola,Moto G Play (2021),Android,32,3,1.42,13,5,5000,6.5,204,159,14/01/2021
1,16,30,2,Motorola,Moto G Play (2021),Android,32,3,1.42,13,5,5000,6.5,204,159,14/01/2021
2,25,30,5,Motorola,Moto G Play (2021),Android,32,3,1.42,13,5,5000,6.5,204,159,14/01/2021
3,32,30,1,Motorola,Moto G Play (2021),Android,32,3,1.42,13,5,5000,6.5,204,159,14/01/2021
4,33,30,10,Motorola,Moto G Play (2021),Android,32,3,1.42,13,5,5000,6.5,204,159,14/01/2021
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
985,237,2,6,Apple,iPhone 13,iOS,128,4,7.75,12,12,3240,6.1,174,699,24/09/2021
986,245,2,6,Apple,iPhone 13,iOS,128,4,7.75,12,12,3240,6.1,174,699,24/09/2021
987,251,2,7,Apple,iPhone 13,iOS,128,4,7.75,12,12,3240,6.1,174,699,24/09/2021
988,256,2,7,Apple,iPhone 13,iOS,128,4,7.75,12,12,3240,6.1,174,699,24/09/2021


In [24]:
df['model'].value_counts()

Moto G Play (2021)      41
Galaxy A32              39
11T Pro                 38
Pixel 6                 38
Moto G Power (2022)     35
Redmi Note 11           35
Poco F4                 34
iPhone 13 Pro Max       34
Galaxy S22 Ultra        32
Galaxy Z Flip 3         32
Nord 2T                 31
Pixel 6a                31
iPhone XR               31
Nord N20                31
Moto G Pure             31
Find X5 Pro             30
Pixel 6 Pro             30
10 Pro                  30
Zenfone 8               30
Galaxy A13              29
iPhone 13 Pro           29
Galaxy A53              29
12 Pro                  28
Galaxy S22              27
Galaxy Z Fold 3         27
iPhone 13               27
Xperia Pro              26
Moto G Stylus (2022)    25
X80 Pro                 24
iPhone 13 Mini          24
Galaxy S22 Plus         22
10T                     20
iPhone SE (2022)        20
Name: model, dtype: int64

In [26]:
df['user_id'].value_counts()

0      10
105    10
30     10
251    10
242    10
       ..
231    10
230    10
208    10
204    10
113    10
Name: user_id, Length: 99, dtype: int64

In [27]:
df.isna().sum()

user_id             0
cellphone_id        0
rating              0
brand               0
model               0
operating system    0
internal memory     0
RAM                 0
performance         0
main camera         0
selfie camera       0
battery size        0
screen size         0
weight              0
price               0
release date        0
dtype: int64

In [28]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 990 entries, 0 to 989
Data columns (total 16 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   user_id           990 non-null    int64  
 1   cellphone_id      990 non-null    int64  
 2   rating            990 non-null    int64  
 3   brand             990 non-null    object 
 4   model             990 non-null    object 
 5   operating system  990 non-null    object 
 6   internal memory   990 non-null    int64  
 7   RAM               990 non-null    int64  
 8   performance       990 non-null    float64
 9   main camera       990 non-null    int64  
 10  selfie camera     990 non-null    int64  
 11  battery size      990 non-null    int64  
 12  screen size       990 non-null    float64
 13  weight            990 non-null    int64  
 14  price             990 non-null    int64  
 15  release date      990 non-null    object 
dtypes: float64(2), int64(10), object(4)
memory u

In [32]:
item_count_pivot = df.pivot_table(index='user_id',columns='model', values= 'rating',aggfunc='sum').fillna(0)
item_count_pivot[item_count_pivot > 0] =1
item_count_pivot.head(10)

model,10 Pro,10T,11T Pro,12 Pro,Find X5 Pro,Galaxy A13,Galaxy A32,Galaxy A53,Galaxy S22,Galaxy S22 Plus,...,Redmi Note 11,X80 Pro,Xperia Pro,Zenfone 8,iPhone 13,iPhone 13 Mini,iPhone 13 Pro,iPhone 13 Pro Max,iPhone SE (2022),iPhone XR
user_id,,,,,,,,,,,,,,,,,,,,,
0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,1.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0
1,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0
6,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
8,1.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
10,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,1.0
12,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
16,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,...,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
24,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,...,0.0,1.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0
25,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0


In [33]:
# Keterangan Olah Dataset
print('Dimensi Dataset :',item_count_pivot.shape)
print('Jumlah Reviewer Item :',item_count_pivot.shape[0])
print('Jumlah Item :',item_count_pivot.shape[1])

Dimensi Dataset : (99, 33)
Jumlah Reviewer Item : 99
Jumlah Item : 33


In [44]:
freq = apriori(item_count_pivot, min_support= 0.1, use_colnames= True)
freq.sort_values('support',ascending=False)

c:\Users\LENOVO\AppData\Local\Programs\Python\Python310\lib\site-packages\mlxtend\frequent_patterns\fpcommon.py:111: DeprecationWarning: DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type
  warnings.warn(


,support,itemsets
13,0.414141,(Moto G Play (2021))
6,0.393939,(Galaxy A32)
2,0.383838,(11T Pro)
20,0.383838,(Pixel 6 )
14,0.353535,(Moto G Power (2022))
...,...,...
141,0.101010,"(Galaxy Z Fold 3, iPhone 13 Pro)"
140,0.101010,"(iPhone 13, Galaxy Z Fold 3)"
139,0.101010,"(Galaxy Z Fold 3, Pixel 6a)"
138,0.101010,"(Galaxy Z Fold 3, Moto G Power (2022))"


In [45]:
rules = association_rules(freq,metric='lift',min_threshold=1)[['antecedents','consequents','support','confidence','lift']]
rules.sort_values(by=['support','confidence','lift'],ascending=False).head(20)

,antecedents,consequents,support,confidence,lift
77,(Galaxy A32),(Moto G Play (2021)),0.212121,0.538462,1.300188
76,(Moto G Play (2021)),(Galaxy A32),0.212121,0.512195,1.300188
18,(11T Pro),(Moto G Play (2021)),0.191919,0.500000,1.207317
19,(Moto G Play (2021)),(11T Pro),0.191919,0.463415,1.207317
34,(iPhone 13 Pro Max),(11T Pro),0.181818,0.529412,1.379257
35,(11T Pro),(iPhone 13 Pro Max),0.181818,0.473684,1.379257
134,(Galaxy Z Flip 3),(Moto G Play (2021)),0.171717,0.531250,1.282774
217,(Redmi Note 11),(Pixel 6 ),0.171717,0.485714,1.265414
81,(Redmi Note 11),(Galaxy A32),0.171717,0.485714,1.232967
216,(Pixel 6 ),(Redmi Note 11),0.171717,0.447368,1.265414
